This program is intended to get latest trending projects on github -- their names, aims and locations.

In [2]:
import re
import csv
import requests
from lxml import etree

In [3]:
url = 'https://github.com/trending'

In [4]:
headers = {'content-type':'application/json'}

In [5]:
r = requests.get(url, headers=headers)
r.status_code

200

In [6]:
html = etree.HTML(r.text)
result = etree.tostring(html)

In [10]:
projects_num = len(html.xpath('//ol[@class="repo-list"]//li//h3/a/@href')) # 根据项目名称数目判断项目总数
projects = []
for i in range(projects_num):
    name_path = '//ol[@class="repo-list"]//li[{}]//h3/a/@href'.format(i+1)
    raw_name = html.xpath(name_path)
    refined_name = raw_name[0]   # TODO 不是很易读，隔了几天就不知道是什么意思了，需要拿到原始材料才能想到
    _, author, project = refined_name.split('/')
    link = 'https://github.com/{author}/{project}'.format(author=author, project=project)
    
    raw_description = html.xpath('//ol[@class="repo-list"]//li[{}]//p//text()'.format(i+1))
    refined_description = max(raw_description, key=len)
    
    star_path = '//ol[@class="repo-list"]//li[{}]//span[@class="d-inline-block float-sm-right"]//text()'.format(i+1)
    raw_star = max(html.xpath(star_path), key=len)
    refined_star = re.search('(\d+)\w+', raw_star.replace(',','')).group(0)
    

    projects.append({'author':author, 'project':project, 'description':refined_description.strip(), 'star':refined_star, 'link':link})
projects

[{'author': 'jlevy',
  'description': 'Master the command line, in one page',
  'link': 'https://github.com/jlevy/the-art-of-command-line',
  'project': 'the-art-of-command-line',
  'star': '1262'},
 {'author': 'jackfrued',
  'description': 'Python - 100天从新手到大师',
  'link': 'https://github.com/jackfrued/Python-100-Days',
  'project': 'Python-100-Days',
  'star': '434'},
 {'author': 'imhuster',
  'description': '中国大陆 31 个省份1978 年至 2019 年一千多万工商企业注册信息，包含企业名称、注册地址、统一社会信用代码、地区、注册日期、经营范围、法人代表、注册资金、企业类型等详细资料。This repository is an dataset of over 10,000,000 enterprise registration data of 31 provinces in Chinese mainland from 1978 to 2019.【工商大数据】、【企业信息】、【enterprise registration data】。',
  'link': 'https://github.com/imhuster/Enterprise-Registration-Data-of-Chinese-Mainland',
  'project': 'Enterprise-Registration-Data-of-Chinese-Mainland',
  'star': '317'},
 {'author': 'entropic-dev',
  'description': 'a package registry for anything, but mostly javascript',
  'link': 'https://github.com/entropi

In [11]:
with open('projects.csv', 'w') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=['author', 'project', 'description', 'star', 'link'])
    writer.writeheader()
    for project in projects:
        writer.writerow(project)

# TODO get stars, links